In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph-Modules")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph-Modules


In [2]:
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_teddynote.tools import GoogleNews
from typing import List, Dict


# 도구 생성
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


@tool
def python_code_interpreter(code: str):
    """Call to execute python code."""
    return PythonAstREPLTool().invoke(code)

In [3]:
from langgraph.prebuilt import ToolNode, tools_condition

# 도구 리스트 생성
tools = [search_news, python_code_interpreter]

# ToolNode 초기화
tool_node = ToolNode(tools)

# ToolNode를 수동으로 호출하기

In [4]:
# 단일 도구 호출을 포함하는 AI 메시지 객체 생성
# AIMessage 객체이어야 함
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "search_news",  # 도구 이름
            "args": {"query": "AI"},  # 도구 인자
            "id": "tool_call_id",  # 도구 호출 ID
            "type": "tool_call",  # 도구 호출 유형
        }
    ],
)

# 도구 노드를 통한 메시지 처리 및 날씨 정보 요청 실행
tool_node.invoke({"messages": [message_with_single_tool_call]})

{'messages': [ToolMessage(content='[{"url": "https://news.google.com/rss/articles/CBMiakFVX3lxTE1yeVNTdUxja0Y0bjlIeDNiZ0JfRzJSVS0wTFhGY01MQnh4Vnd3a1NLRVNMQXdzY1V6NXVFZlUyeGp4NjBJSF9aWHh3Y0xqYTQ3TndSdUhfVUxjTUlYQ19qTTdfcDh2c0kyclE?oc=5", "content": "네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것” - AI타임스"}, {"url": "https://news.google.com/rss/articles/CBMiVkFVX3lxTE8zYjdSRmMtbXJ4RmJXRlJPa2hWblBTS01oT1ltdGJJbzlSSzZNMlFxVG9uZVdqbEpQWWt5eXRIdWlaR2JUSzNwMk9yb0FBUVRERmdvN3VB?oc=5", "content": "부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟 - 지디넷코리아"}, {"url": "https://news.google.com/rss/articles/CBMiW0FVX3lxTE1kaXFoV3RsZUZfQXdRR21pTWVrRURMMjZQVmNrbmVZWUhiR2JGUGJxNWZwcE0za3kyYVR5QTQtWXZ0UDBpMzA2M3F5UGR1WnNGbkh6alFIT3dJZ2_SAWBBVV95cUxQNS00UjZ3SUFYVnFmU01PYUhmWGxiMV81RlQzT0psa3BNUlE4cmFJZU91cW40Q1lrbU93R3NGSEh3WnBNeXQ2QXo3azUta0ozM2RfTTUtZUpfOFRYbGhDY2s?oc=5", "content": "미일 산학 협력 AI연구 거점, 日쓰쿠바대에 이르면 내년말 개설 - 연합뉴스"}, {"url": "https://news.google.com/rss/articles/CBMiU0FVX3lxTFBqZHVQSXRHMWhRRW1wNUMtajItL

In [5]:
# 다중 도구 호출을 포함하는 AI 메시지 객체 생성 및 초기화
message_with_multiple_tool_calls = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "search_news",
            "args": {"query": "AI"},
            "id": "tool_call_id",
            "type": "tool_call",
        },
        {
            "name": "python_code_interpreter",
            "args": {"code": "print(1+2+3+4)"},
            "id": "tool_call_id",
            "type": "tool_call",
        },
    ],
)

# 생성된 메시지를 도구 노드에 전달하여 다중 도구 호출 실행
tool_node.invoke({"messages": [message_with_multiple_tool_calls]})

{'messages': [ToolMessage(content='[{"url": "https://news.google.com/rss/articles/CBMiakFVX3lxTE1yeVNTdUxja0Y0bjlIeDNiZ0JfRzJSVS0wTFhGY01MQnh4Vnd3a1NLRVNMQXdzY1V6NXVFZlUyeGp4NjBJSF9aWHh3Y0xqYTQ3TndSdUhfVUxjTUlYQ19qTTdfcDh2c0kyclE?oc=5", "content": "네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것” - AI타임스"}, {"url": "https://news.google.com/rss/articles/CBMiVkFVX3lxTE8zYjdSRmMtbXJ4RmJXRlJPa2hWblBTS01oT1ltdGJJbzlSSzZNMlFxVG9uZVdqbEpQWWt5eXRIdWlaR2JUSzNwMk9yb0FBUVRERmdvN3VB?oc=5", "content": "부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟 - 지디넷코리아"}, {"url": "https://news.google.com/rss/articles/CBMiW0FVX3lxTE1kaXFoV3RsZUZfQXdRR21pTWVrRURMMjZQVmNrbmVZWUhiR2JGUGJxNWZwcE0za3kyYVR5QTQtWXZ0UDBpMzA2M3F5UGR1WnNGbkh6alFIT3dJZ2_SAWBBVV95cUxQNS00UjZ3SUFYVnFmU01PYUhmWGxiMV81RlQzT0psa3BNUlE4cmFJZU91cW40Q1lrbU93R3NGSEh3WnBNeXQ2QXo3azUta0ozM2RfTTUtZUpfOFRYbGhDY2s?oc=5", "content": "미일 산학 협력 AI연구 거점, 日쓰쿠바대에 이르면 내년말 개설 - 연합뉴스"}, {"url": "https://news.google.com/rss/articles/CBMiU0FVX3lxTFBqZHVQSXRHMWhRRW1wNUMtajItL

# llm 과 함께 사용하기

In [7]:
from langchain_openai import ChatOpenAI

# LLM 모델 초기화 및 도구 바인딩
model_with_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

In [8]:
# 도구 호출 확인
model_with_tools.invoke("처음 5개의 소수를 출력하는 python code 를 작성해줘").tool_calls

[{'name': 'python_code_interpreter',
  'args': {'code': 'def first_n_primes(n):\n    primes = []\n    num = 2  # Starting from the first prime number\n    while len(primes) < n:\n        is_prime = True\n        for i in range(2, int(num**0.5) + 1):\n            if num % i == 0:\n                is_prime = False\n                break\n        if is_prime:\n            primes.append(num)\n        num += 1\n    return primes\n\n# Get the first 5 prime numbers\nfirst_n_primes(5)'},
  'id': 'call_V3KilnXj3EYQFjwwyVKk6m9o',
  'type': 'tool_call'}]

In [9]:
# 도구 노드를 통한 메시지 처리 및 LLM 모델의 도구 기반 응답 생성
tool_node.invoke(
    {
        "messages": [
            model_with_tools.invoke(
                "처음 5개의 소수를 출력하는 python code 를 작성해줘"
            )
        ]
    }
)

{'messages': [ToolMessage(content='[2, 3, 5, 7, 11]', name='python_code_interpreter', tool_call_id='call_ENEMTOw1tKu74jMgiY3oAW7z')]}

# Agent 와 함께 사용하기

In [10]:
# LangGraph 워크플로우 상태 및 메시지 처리를 위한 타입 임포트
from langgraph.graph import StateGraph, MessagesState, START, END

# LLM 모델을 사용하여 메시지 처리 및 응답 생성, 도구 호출이 포함된 응답 반환
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

# 메시지 상태 기반 워크플로우 그래프 초기화
workflow = StateGraph(MessagesState)

# 에이전트와 도구 노드 정의 및 워크플로우 그래프에 추가
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# 워크플로우 시작점에서 에이전트 노드로 연결
workflow.add_edge(START, "agent")

# 에이전트 노드에서 조건부 분기 설정, 도구 노드 또는 종료 지점으로 연결
workflow.add_conditional_edges("agent", tools_condition)

# 도구 노드에서 에이전트 노드로 순환 연결
workflow.add_edge("tools", "agent")

# 에이전트 노드에서 종료 지점으로 연결
workflow.add_edge("agent", END)


# 정의된 워크플로우 그래프 컴파일 및 실행 가능한 애플리케이션 생성
app = workflow.compile()

In [11]:
# 실행 및 결과 확인
for chunk in app.stream(
    {"messages": [("human", "처음 5개의 소수를 출력하는 python code 를 작성해줘")]},
    stream_mode="values",
):
    # 마지막 메시지 출력
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

처음 5개의 소수를 출력하는 python code 를 작성해줘
================================== Ai Message ==================================
Tool Calls:
  python_code_interpreter (call_5PPMjLLnSXqaF2dJjDsWyJXR)
 Call ID: call_5PPMjLLnSXqaF2dJjDsWyJXR
  Args:
    code: def first_n_primes(n):
    primes = []
    num = 2  # Starting from the first prime number
    while len(primes) < n:
        is_prime = True
        for i in range(2, int(num**0.5) + 1):
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(num)
        num += 1
    return primes

# Get the first 5 prime numbers
first_n_primes(5)
================================= Tool Message =================================
Name: python_code_interpreter

[2, 3, 5, 7, 11]
================================== Ai Message ==================================

처음 5개의 소수는 다음과 같습니다: [2, 3, 5, 7, 11]


In [12]:
# 검색 질문 수행
for chunk in app.stream(
    {"messages": [("human", "search google news about AI")]},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

search google news about AI
================================== Ai Message ==================================
Tool Calls:
  search_news (call_FOalPYTdnFDgzlRKGpaEJ8ub)
 Call ID: call_FOalPYTdnFDgzlRKGpaEJ8ub
  Args:
    query: AI
================================= Tool Message =================================
Name: search_news

[{"url": "https://news.google.com/rss/articles/CBMiakFVX3lxTE1yeVNTdUxja0Y0bjlIeDNiZ0JfRzJSVS0wTFhGY01MQnh4Vnd3a1NLRVNMQXdzY1V6NXVFZlUyeGp4NjBJSF9aWHh3Y0xqYTQ3TndSdUhfVUxjTUlYQ19qTTdfcDh2c0kyclE?oc=5", "content": "네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것” - AI타임스"}, {"url": "https://news.google.com/rss/articles/CBMiVkFVX3lxTE8zYjdSRmMtbXJ4RmJXRlJPa2hWblBTS01oT1ltdGJJbzlSSzZNMlFxVG9uZVdqbEpQWWt5eXRIdWlaR2JUSzNwMk9yb0FBUVRERmdvN3VB?oc=5", "content": "부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟 - 지디넷코리아"}, {"url": "https://news.google.com/rss/articles/CBMiW0FVX3lxTE1kaXFoV3RsZUZfQXdRR21p

In [13]:
# 도구 호출이 필요 없는 질문 수행
for chunk in app.stream(
    {"messages": [("human", "안녕? 반가워")]},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

안녕? 반가워
================================== Ai Message ==================================

안녕하세요! 반갑습니다. 어떻게 도와드릴까요?
